## Kernel

<hr>

### 왜 3x3 컨볼루션이 대세가 되었는가?
- 이전에는 큰 이미지에는 큰 Filter로, 작은 이미지에는 보다 작은 Filter로 적용하는 것이 관례 였다고 함.

- **VGGNet에서는 5x5 하나도 가능하지만, 3x3 filter를 2개 Layer에 걸쳐 사용해도 커버하는 영역이 똑같다.**
    - 즉 작은 필터(3*3)을 쓰더라도, Layer를 겹쳐서 여러 개의 필터를 거치면, 큰 Feature를 뽑아낼 수 있다.
    
- **Convolution Layer가 늘어날 수록 뒤에 있는 Layer가 더 고차원적인 Feature를 뽑아낸다.**
    - 첫번째 Layer의 필터들에서는 대게 선형의 Feature들을 뽑아내고
    - 두번쨰 Layer의 필터들에서는 원형의 Feature들
    - 세번째 Layer의 필터들은 디테일한 Feature들을 뽑아내는 것이 가능
    - ==> 5x5보다 3x3필터 2개가 더 성능이 좋을 것이다.

### 5x5 vs 3x3 가중치 비교
- 5x5 필터는 weight 의 갯수는 25개
- 3x3 필터를 2개 겹치게 되면 9개씩 2개 => 18개

### Kernel 관련 큰 축 중 하나인 VGGNet의 결론
- 1) 5x5보다 3x3이 Weight가 더 적다 => 메모리 효율 높고, 실행속도 높다
- 2) 시각화 해보니 Layer가 많으면 고차원적인 이미지를 뽑아낸다 


## Padding 사용하는 이유
- **Convolution 을 진행하다 보면 원본이미지보다 작아진다 ==> 즉 이미지를 Convolution연산을 해야하는데 이미지가 작아지다보니까 원본 이미지로 진행을 못할 수가 있다.**

## Stride 값 선택
- **stride 는 1이 best다**
- stride가 1이여야 가장 디테일한 Feature들을 뽑아낼 수 있음

### 예외적인 Stride값 사용
- 맥스풀링은 필요없는 Feature를 버리는 개념이기 때문에 Maxpooling을 적용하는 것은 좋은 작업.
- **예외적으로 Stride를 2를 사용하는 경우 == Maxpooling 을 대체할 때 사용**
    - Maxpooling Layer에 Conv2D 연산을 집어넣는 것.
- Maxpooling 없애고 Convolution 넣었을 때 장점
    - Maxpooling 있을 때보다 연산횟수는 줄어듬 (3x3 커널 또 쓸꺼고 연산횟수는 줄어들음)
    - Maxpooling보다 좀 더 디테일하게 Feature를 뽑아냄.
    
- 단점
    - Conv Layer 추가로 인해 그만큼 filter가 생겨나 weight의 전체 수가 증가

- 사용사례
    - 이미지 제너레이터 문제에서는 필요없다고 상대적으로 약한 Feature를 버려버리면, 추후 이미지 생성시 유사한 이미지를 만들지 못하고, 부족한 Feautre 정보 일 수 있다.
    - 즉 이미지 제너레이터에서는 Maxpooling 대신 Conv2D의 Stride(2,2) 로 해야함.
    
## Layer와 Filter 수에 대한 Trade-off
- ResNet-152 Layer경우보다 < 50개의 Layer + Filter수 증가한 Wide Resnet이 성능이 더 좋았다.
- 즉 어떤 base 모델이 있다고 가정하였을 때, Layer를 1/2로 줄이는 한이 있더라도 Filter수를 2배 더 가져가는 것이 성능이 좋아질 수 있다.

## ResNet 에서의 1x1 Filter

### 1x1 Conv Layer를 쓰는 장점 3가지
- bottle neck 전략
    - 256개의 input filter를 64개가 나오도록 (1,1)Filter로 압축
    - 중간에서 Conv(3,3) 연산
    - 64개의 input filter를 256개가 나오도록 (1,1)Filter로 함
    - **즉 (3,3) Conv Layer 2개 < (1,1) encoding ,(3,3) Conv, (1,1) decoding**
- bottleneck 전략 장점
    - (1,1) filter를 이용한 BottleNeck 전략은 input_filter를 4배 더 많이 넣을 수 있다
    - 연산도 더 효율적이다(weight 갯수가 더 작다)
        - 64x3x3x64 + 64x3x3x64 => weight 73728개
        - 256x1x1x64 + 64x3x3x64 + 64x1x1x256 => weight 69632개

- (1,1) Conv 연산의 장점은 fully-connected를 대체할 수 있다는 점
    - FC를 (1,1) Conv Layer로 대체하고, weight가 줄어든 만큼 filter를 늘리거나 layer를 더 쌓는 trade-off 가능

- Dense 전략 
    - (기존 FC는 input 이미지의 size가 정확히 모델과 일치 해야 한다
    - Conv.Layer는 input 사이즈가 중요하지 않고, 전부 (1,1) Conv Layer로 대체시 input 이미지의 사이즈가 자유로워짐
    - test시에 좀 더 큰 이미지를 넣어 결과를 내주면 똑같은 train모델에 대해 더 좋은 성능을 가진다. (이미지 피쳐를 더 뽑을 수 있으니까)
    

## 참고문헌
- https://nittaku.tistory.com/266